<a href="https://colab.research.google.com/github/yjodev/Tobigs19/blob/main/week3_DT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [3]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [8]:
pd_data.shape

(14, 5)

## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [9]:
def get_gini(df, label):
    
    n = df.shape[0] # 데이터 수
    
    # label 값
    label_value = np.unique(df[label])
    
    # 첫번째 레이블 수
    label1 = np.sum(df[label] == label_value[0])
    
    # 두번째 레이블 수(binary classification이므로 첫 번째 라벨이 아니면 두번째 라벨)
    label2 = n - label1 
    
    # 지니 계수 계산식
    gini = 1 - (label1 / n) ** 2 - (label2 / n) ** 2
        
    return gini

In [10]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [20]:
print(pd_data["age"].unique())
print(len(pd_data["age"].unique()))

['youth' 'middle_aged' 'senior']
3


In [24]:
for i in range(3,5):
  print(i)

3
4


In [25]:
from itertools import combinations

def get_binary_split(df, attribute): # 이진 분류 만들기
    
    uniques = list(df[attribute].unique()) # attribute열의 고유값 리스트 
    
    result = [] # 결과값 리스트 초기화

    for i in range(1, len(uniques)): # 1부터 attribute 열의 고유값의 개수-1만큼 반복 
        for att in combinations(uniques, i):
            result.append(list(att))
        # combination: 하나의 리스트에서 모든 조합 구하기. 조합의 원소의 수는 1부터 선택한 열의 고유값 개수 -1까지 반복
        
    
    return result

In [28]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [37]:
for splits in get_binary_split(pd_data, "income"):
  a = np.array(list(map(lambda x : x in splits, pd_data["income"])))
  print(a)

[ True  True  True False False False False False False False False False
  True False]
[False False False  True False False False  True False  True  True  True
 False  True]
[False False False False  True  True  True False  True False False False
 False False]
[ True  True  True  True False False False  True False  True  True  True
  True  True]
[ True  True  True False  True  True  True False  True False False False
  True False]
[False False False  True  True  True  True  True  True  True  True  True
 False  True]


In [41]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {} # 결과 리스트 초기화
    binary_split = get_binary_split(df, attribute) # 열의 고유값으로 이진 분류
    n = len(df) # 데이터 개수
    
    for splits in binary_split: # 이진분류한 조합들마다 반복하라
        split_key = "_".join(splits) # dictionary의 key값 (2개 이상이면 _로 구분)
        
        mask = np.array(list(map(lambda x : x in splits, df[attribute])))
        # lambda x: x in splits : x값이 splits와 같은지 T/F
        # list(map( 함수 , 리스트 )): 리스트로부터 원소를 하나씩 꺼내어 함수값 계산 후 함수값을 새로운 리스트에 담음
        # 뒤에 나오는 뒤집기 연산 (~)을 위해 numpy array 형식으로 저장

        # 원하는 DataFrame만 뽑아주기(t = target, r = rest)
        data_t = df.loc[mask, :]
        data_r = df.loc[~mask, :]        

        # gini계수 계산
        gini_t = get_gini(data_t, label)
        gini_r = get_gini(data_r, label)
        
        gini = len(data_t) / n * gini_t + len(data_r) / n * gini_r
        
        # dictionary의 value값에 대응하여 저장
        result[split_key] = gini
    
    return result

In [40]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.44285714285714295,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.44285714285714295}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [42]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.44285714285714295)

In [43]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [47]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    lowest_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f"Minimum Gini index of {feature} : {lowest_gini:.4f}")

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [48]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [49]:
from math import log2

def getEntropy(df, feature) :
    
    entropy = 0 # for문 돌기 전 entropy 초기화
    p_is = df[feature].value_counts() / len(df) 
    # value_counts함수를 이용하여 feature의 class별 확률 p_i를 계산

    for p_i in p_is:
        entropy -= p_i * np.log2(p_i)
        # entropy에 각각의 i별로 p * log2(p)를 구한값을 빼서 업데이트
    return entropy


In [50]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [57]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {} # 빈 dictionary로 초기화

    # 타겟 변수 제외한 엔트로피 계산
    def getEntropy_lower(df, feature, other_feature):
        entropy = 0 # entropy값 0으로 초기화
        n = len(df) # 데이터의 개수
        unique = df[other_feature].unique()
        # other_feature의 고유값을 저장
        
        for i in unique: # other_feature의 고유값별로 반복
            tmp_df = df[df[other_feature].isin([i])] 
            # isin함수를 사용하여 other_feature의 class가 i인 것만 저장
            entropy += len(tmp_df) / n * getEntropy(tmp_df, feature)
            # 개별 엔트로피 계산 함수인 getEntropy함수를 통해 target feature의 entropy를 가중평균함
        return entropy
        
    info_Target = getEntropy(df, feature) # 타겟 Feature에 대한 Entropy를 구함
    other_features = list(df.loc[:, df.columns != feature])
    # 타겟변수를 제외한 나머지 설명변수들을 리스트 형태로 저장
    
    for other_feature in other_features:
        result[other_feature] = info_Target - getEntropy_lower(df, feature, other_feature)
        # getEntropy_lower함수를 이용하여 타겟 feature 제외한 feature의 entropy를 계산한 후 info_Target에서 빼서 information gain을 계산
    return result

In [58]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}